<a href="https://colab.research.google.com/github/PadmajaVB/Sexist-Statement-Detection/blob/main/gettweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tweepy

# Twitter Developer keys here
# USED TO COMMUNICATE TO THE TWITTER API
consumer_key = 'DYblLECW6ZxgdNmZbOL5Vsbty'
consumer_key_secret = '5bjrh8S8rNXEVQSMDUl9DX3yYtqXSFFJWJ8r7mwDFgs0cUSF6P'
access_token = '1089903230992056320-rpusTvBh7re56F64sj7YoTCeMLrDcb'
access_token_secret = '1ksc9eaJer3c5cQc6HYVG5UCqkPdL8I3Eb1Oz1dQxieh9'


In [ ]:
#set up the API for fetching
auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
# This is corpus dataset
import pandas as pd
#tweet id form (check this tsv for format)
corpusFile = "benevolent_sexist.tsv"
df= pd.read_csv(corpusFile, header= None)


In [ ]:
# This method creates the training set
def createTrainingSet(corpusFile):
    import csv
    import time

    c=0
    #print(corpusFile.head())
    #Get each id as part of the list
    corpus = [id for id in corpusFile[0]]

    

    sleepTime = 4
    trainingDataSet = []
    #Do not send more than 1500 ids at once else you will have to wait for 1 hour to cooldown
    for tweet in corpus:
        try:
            tweetFetched = api.get_status(tweet) #get the metadeta
            print(c,"Tweet fetched" + tweetFetched.text)
            c+=1 #counter for seeing which tweet index is being fetched
            time.sleep(sleepTime) #for the API to cool down
            trainingDataSet.append(tweetFetched.text)
            

        except Exception as e:
            print(c,"Inside the exception - no:2")
            print(e)
            c+=1
            print()
            time.sleep(2)
            continue
    #return in list       
    return trainingDataSet
    


In [ ]:
resultFile = createTrainingSet(df)

In [ ]:
res= pd.DataFrame(resultFile)

In [ ]:
res.to_csv('samplename.csv')

In [ ]:
print(res.loc[2][0])

RT @ConstanceQueen8: #ADayWithoutWomen Trump Women
Think Differently
Don't Neglect Responsibilities 
Don't Whine &amp; Complain
Recognize &amp; Che…
